In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20398213
paper_name = 'ruiz_roig_de_nadal_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 277 x 4


In [7]:
original_data.head()

,ORF code,Gene name,Gene description,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,YAL021C,CCR4,Transcriptional regulator,-3.0
2,YBL007C,SLA1,Cytoskeleton assembly control protein,-3.0
3,YBL024W,NCL1,tRNA (cytosine-5-)-methyltransferase,-3.0
4,YBL058W,SHP1,Potential regulatory subunit for Glc7p,-3.0


In [8]:
original_data['orf'] = original_data['ORF code'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            ORF code Gene name Gene description  Unnamed: 3  orf
index_input                                                     
0                NaN       NaN              NaN         NaN  NAN


In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data['data'] = original_data['Unnamed: 3'].astype(float)

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(276, 1)

# Load & process tested strains

In [20]:
tested = pd.read_excel('raw_data/KO_386.xlsx', sheet_name='Sheet1', skiprows=2)

In [21]:
tested.head()

,Unnamed: 0,Unnamed: 1,Plate,row,col,record #,ORF,strain,batch,row.1,col.1,Euroscarf comments,Freezing comments,Duplication in our collection,SL/wt: Boone,gene name(s),brief id (fromSGD),notes about QC,QC results,comments about QC
0,1+2,NaN,1,A,1,338.0,YAL068C,BY4741,chr1_1,A,2.0,NaN,NaN,NaN,NaN,NaN,"biological_process unknown, molecular_function...",NaN,NaN,NaN
1,1+2,NaN,1,A,2,339.0,YAL067C,BY4741,chr1_1,A,3.0,NaN,NaN,NaN,NaN,SEO1,putative permease,NaN,NaN,NaN
2,1+2,1,1,A,3,340.0,YAL066W,BY4741,chr1_1,A,4.0,NaN,NaN,NaN,NaN,NaN,"biological_process unknown, molecular_function...",NaN,NaN,NaN
3,1+2,NaN,1,A,4,341.0,YAL065C,BY4741,chr1_1,A,5.0,NaN,NaN,NaN,NaN,NaN,"biological_process unknown, molecular_function...",NaN,NaN,NaN
4,1+2,NaN,1,A,5,345.0,YAL062W,BY4741,chr1_1,A,9.0,NaN,NaN,NaN,NaN,"GDH3, FUN51","NADP-linked glutamate dehydrogenase,glutamate ...",NaN,NaN,NaN


In [22]:
tested['orf'] = tested['ORF'].astype(str)

In [23]:
tested['orf'] = clean_orf(tested['orf'])

In [24]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

            Unnamed: 0 Unnamed: 1  Plate row  col  record #  ORF strain batch  \
index_input                                                                     
756              15+16        NaN     16   H    1       NaN  NaN    NaN   NaN   
757              15+16        NaN     16   H    2       NaN  NaN    NaN   NaN   
758              15+16        NaN     16   H    3       NaN  NaN    NaN   NaN   
759              15+16        NaN     16   H    4       NaN  NaN    NaN   NaN   
760              15+16        NaN     16   H    5       NaN  NaN    NaN   NaN   
761              15+16        NaN     16   H    6       NaN  NaN    NaN   NaN   
762              15+16        NaN     16   H    7       NaN  NaN    NaN   NaN   
763              15+16        NaN     16   H    8       NaN  NaN    NaN   NaN   
764              15+16        NaN     16   H    9       NaN  NaN    NaN   NaN   
765              15+16        NaN     16   H   10       NaN  NaN    NaN   NaN   
766              15+16      

In [26]:
tested = tested.loc[t,:]

In [27]:
tested_orfs = tested['orf'].unique()

In [28]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [15983]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,15983
data_type,value
orf,
YAL068C,0.0
YAL067C,0.0
YAL066W,0.0
YAL065C,0.0
YAL062W,0.0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,15983
,data_type,value
gene_id,orf,
1869,YAL068C,0.0
61,YAL067C,0.0
60,YAL066W,0.0
1727,YAL065C,0.0
57,YAL062W,0.0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      15983       
data_type       value valuez
gene_id orf                 
1869    YAL068C   0.0    0.0
61      YAL067C   0.0    0.0
60      YAL066W   0.0    0.0
1727    YAL065C   0.0    0.0
57      YAL062W   0.0    0.0

# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 20398213...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.05s/it]

Updating the data_modified_on field...
